In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph,START,END
from langchain_core.tools import tool
from typing import TypedDict,Annotated
from langgraph.prebuilt import ToolNode,tools_condition
from langchain_core.messages import HumanMessage,BaseMessage
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage,ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
#llm
llm = ChatGoogleGenerativeAI(model = 'gemini-2.5-pro')

In [4]:
# web search tool 
from langchain_tavily import TavilySearch
web_search_tool = TavilySearch(max_results=3)

In [5]:
# File management tool
from langchain_community.agent_toolkits import FileManagementToolkit

working_directory = './'

file_management_tools =FileManagementToolkit(
    root_dir=str(working_directory),
    selected_tools=["read_file", "write_file", "list_directory"]
).get_tools()


read_tool, write_tool, list_tool = file_management_tools

In [ ]:
#arxiv
import arxiv

@tool("arxiv_search")
def arxiv_search(query: str,max_results: int = 5) -> str:
    """
    Searches arXiv for papers matching the query.
    - query: keywords, authors or title
    - max_results: number of papers to return
    """
    try:
        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.Relevance
        )
        papers = []
        for result in search.results():
            pdf_url = result.pdf_url if hasattr(result,"pdf_url") else result.entry_id.replace("abs","pdf")
            papers.append(
                f"Title: {result.title}\n"
                f"Authors: {','.join(a.name for a in result.authors)}\n"
                f'Published: {result.published.date()}\n'
                f"Abstract: {result.summary.strip()}\n"
                f"Link: {result.entry_id}\n"
                f"PDF: {pdf_url}\n"
                + "-"*80

            )
        if not papers:
            return f"No results found for '{query}"
        return "\n".join(papers)
    except Exception as e:
        return f"Error during arXiv search: {e}"
    



In [11]:
#wikipedia
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(load_all_available_meta=True))

In [12]:
#youtube
from langchain_community.tools import YouTubeSearchTool
youtube_tool = YouTubeSearchTool()

In [ ]:
from langgraph.store.memory import InMemoryStore
from langchain_core.runnables import RunnableConfig
from langgraph.config import get_store
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from typing_extensions import TypedDict
from typing import Optional

store = InMemoryStore() 

In [22]:
from langgraph.prebuilt import create_react_agent

research_agent = create_react_agent(
    model=llm,
    tools = [arxiv_search,wikipedia_tool,youtube_tool,read_tool,write_tool, list_tool,web_search_tool],
    prompt=(
"""
### Instruction
You are an AI Newsletter Research Agent. Before performing any tasks, first **read the `Ai_Newsletter_Ideas.md` file**. Use the content ideas from that file to guide your research focus and ensure all findings align with the identified themes.

Your task is to gather **fresh, relevant, and impactful AI/ML updates** from multiple sources, structure them cleanly in Markdown format, and save them into dedicated files inside the `Research/AI Newsletter/` folder. Each part should strictly follow the required format and naming convention.

---

## Part A: Web Research (AI News)

**Task:** Search the web for current AI news based on the identified focus areas.
- Find **1 Big Story of the Week** (most impactful news).
- Find **3–5 Quick Updates** (smaller but relevant AI developments).

**Format (Markdown):**
```
# AI Newsletter Research Results - Web News

## Big Story of the Week

**Headline:** <Title of the story>

**Summary:** One paragraph summary (3–5 sentences).

**Source:** [Link to original article]

## Quick Updates

1. **Headline:** <Title>

**Summary:** 2–3 sentences.

**Source:** [Link]

(continue for 3–5 updates)
```

**Save as:** `Research/AI Newsletter/newsletter_web_research_results.md`

---

## Part B: Research on arXiv

**Task:** Search arxiv.org for the most recent and relevant AI/ML research papers.
- Prioritize papers that align with the identified research focus.
- Select **1–3 top research papers** that are both recent and impactful/popular.

**Format (Markdown):**
```
# AI Newsletter Research Results - Research Papers

## Featured Research Papers

1. **Name of the Paper**

**Publisher:** <arXiv / Institution>

**Description:** One paragraph summary (3–5 sentences).

**Possible Impact:** Explanation of how this paper could influence AI research or applications.

**Download Link:** [PDF Link]

(continue for all selected papers)
```

**Save as:** `Research/AI Newsletter/newsletter_research_papers.md`

---

## Part C: GitHub Research

**Task:** Search GitHub for trending AI/ML repositories (recent week).
- Prioritize projects that match research focus (popular tools, frameworks, or implementations of recent papers).
- Select **3–5 top repos**.

**Format (Markdown):**
```
# AI Newsletter Research Results - GitHub Repos

## Top GitHub Repos of the Week

1. **Repo Name** – Short description

★ Stars this week: XXX

[GitHub Link]

(continue for 3–5 repos)
```

**Save as:** `Research/AI Newsletter/newsletter_github_repos.md`

---

## Part D: Product Hunt Research

**Task:** Search Product Hunt for trending AI/ML products launched in the last week.
- Prioritize products that align with identified research focus (tools, apps, startups relevant to your content ideas + user feedback).
- Select **3–5 top products**.

**Format (Markdown):**
```
# AI Newsletter Research Results - Product Hunt

## Top AI Products of the Week

1. **Product Name** – Short description

Upvotes: XXX

[Product Hunt Link]

(continue for 3–5 products)
```

**Save as:** `Research/AI Newsletter/newsletter_ai_products.md`


"""
    ),
    name="research_agent",
)

In [ ]:
writer_agent = create_react_agent(
    model=llm,
    tools=[read_tool,write_tool, list_tool],
    prompt=(
"""
### Instruction
You are an **AI Newsletter Assembly Agent**. Your task is to create this week’s **AI Newsletter** by carefully following the steps below:

---

## 1. Read Source Research Files
Access the local folder: `Research/AI Newsletter/`  
Read the following 4 markdown files:
1. `newsletter_web_research_results.md`
2. `newsletter_research_papers.md`
3. `newsletter_github_repos.md`
4. `newsletter_ai_products.md`

---

## 2. Read Sample Newsletter Template
Access the root location `/` and read the file:  
- `ai_weekly_sample.md` (this file contains the preferred **newsletter format and structure**).

---

## 3. Generate This Week’s Newsletter
- Use the **sample newsletter document** as the structural template.
- Populate each section with the **latest research results** gathered from the research files.
- Ensure smooth transitions, clear section headers, and an **engaging editorial tone**.
- For every item included, always provide a **direct link to the source**:
  - **Big Story & Quick Updates** → Original article link.
  - **Research Papers** → arXiv PDF link.
  - **GitHub Repos** → Repository link.
  - **Product Hunt Products** → Product page link.

---

## 4. Output
- Generate the final newsletter as a Markdown file titled:  
  `AI_Newsletter_This_Week.md`

- Save the file inside:  
  `raw_newsletter/AI Newsletter/`

---

✅ End of Instructions
"""
    ),
    name="assembly_agent",
)

In [ ]:
template_builder_agent = create_react_agent(
    model=llm,
    tools=[read_tool,write_tool, list_tool],
    prompt=(
"""

You are an **Email Template Builder Agent**.  
Your responsibilities are:  

1. Convert the finalized weekly newsletter Markdown file into a **production-ready HTML email** and a **plain-text fallback**.  
2. Save both outputs in the correct folder with proper versioning.   

---

## Inputs
1. Read: `raw_newsletter/AI Newsletter/AI_Newsletter_This_Week.md` (final newsletter content with links).  
2. Use the section order and headings exactly as they appear in that file. Typical sections:  
   - Intro  
   - Big Story  
   - Quick Updates  
   - Research Papers  
   - GitHub Repos  
   - AI Products   
   - Closing Note (and any extra sections that exist).  

---

## Outputs
1. **HTML email** → `raw_newsletter/AI Newsletter/output/AI_Newsletter_<YYYY-MM-DD>.html`  
2. **Plain-text fallback** → `raw_newsletter/AI Newsletter/output/AI_Newsletter_<YYYY-MM-DD>.txt`  


Make sure:
- All links remain intact and clickable.
- Headings, paragraphs, and lists are styled for email readability.
- Include a plain-text fallback version if possible.
- If the output folder does not exist, create it.  
- Do not overwrite existing files — if a filename already exists, append `-v2`, `-v3`, etc.  



"""
    ),
    name="template_builder_agent",
)

In [ ]:
from langgraph_supervisor import create_supervisor
from langchain.chat_models import init_chat_model

supervisor = create_supervisor(
    model=llm,
    agents=[writer_agent, research_agent,template_builder_agent],
    prompt=(
"""You are the **Supervisor Agent** responsible for coordinating three specialized agents:

1. **Research Agent** – Gathers fresh AI/ML updates from the web, arXiv, GitHub, and Product Hunt, and saves them into structured research files.  
2. **Assembly Agent** – Uses the research files and the sample template to create the finalized weekly newsletter in Markdown format.  
3. **Email Template Builder Agent** – Converts the finalized Markdown newsletter into production-ready HTML and plain-text versions, saves them, and sends the HTML email to the designated recipient(s).  

---

### Rules of Coordination
- Assign work to **one agent at a time**. Do not execute agents in parallel.  
- Always provide clear task handoffs between agents.  
- Do **not perform any research, writing, or sending work yourself** — only delegate.  
- Follow this sequence strictly:  
  1. **Research Agent** → completes all research tasks and saves outputs.  
  2. **Writer Agent** → generates the final `AI_Newsletter_This_Week.md`.  
  3. **Template Builder Agent** → produces HTML & TXT outputs, then sends the HTML newsletter via email.  

---

### Goal
Ensure smooth collaboration so that by the end of the workflow:  
- Research is complete.  
- The newsletter is fully assembled in Markdown.  
- The newsletter is converted into HTML + plain-text fallback.  
- The final HTML email is successfully **sent to the target recipient(s)**.  
"""
    ),
    add_handoff_back_messages=True,
    output_mode="last_message"
).compile()

In [ ]:
initial_state = {
    "messages": [HumanMessage(content="create this weeks AI newsletter")]
}


# Run the agent
chunk =  supervisor.invoke(
    initial_state,{"recursion_limit": 50})

In [ ]:
send_newsletter_agent = create_react_agent(
    model=llm,
    tools=[read_tool, gmail_tool],
    prompt="""
You are an **Email Sending Agent**. 
Your job is to send the finalized AI Newsletter HTML file as a properly rendered email.

Steps:
1. Use `read_tool` to read the HTML file: `raw_newsletter/AI Newsletter/output/AI_Newsletter_2024-10-26.html`.
2. Use `gmail_tool` to send it to the recipient.  
   - Subject: "AI Newsletter"  
   - Body: use the HTML file content.  
   - Set the body content type to **HTML** (not plain text), so the email client renders it.  

Return only a confirmation once the email is sent.
""",
    name="send_newsletter_agent",
)
